# Build Train Feature Table V4 (Strategic Features)

이 노트북은 `kkbox_train_feature_v3.parquet`를 기반으로, 모델 보고서에서 제안된 **전략적 파생 변수(Strategic Derived Features)**를 추가하여 V4 데이터셋을 생성합니다.

## 추가되는 피처 (New Features)
1. **`active_decay_rate`**: 최근 활동 감소율 (w7 vs w30)
2. **`listening_time_velocity`**: 청취 시간 가속도 (w7 - w14)
3. **`discovery_index`**: 탐색 지수 (Unique 곡 비중)
4. **`skip_passion_index`**: 스킵 열정도 (25% 미만 / 100% 청취)
5. **`last_active_gap`**: 마지막 접속 경과일 (잠수 유저 포착) - *Raw Log 처리 필요*
6. **`daily_listening_variance`**: 청취 루틴 안정성 -> 기존 `std_secs_w7` 활용

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

pd.set_option('display.max_columns', None)

In [ ]:
# 경로 설정
if os.getcwd().endswith("preprocess"):
    os.chdir("../..")
    
DATA_DIR = "data/processed"
RAW_DATA_DIR = "data/raw"
V3_PATH = os.path.join(DATA_DIR, "kkbox_train_feature_v3.parquet")
USER_LOGS_PATH = os.path.join(RAW_DATA_DIR, "user_logs_v2.csv")
OUTPUT_PATH = os.path.join(DATA_DIR, "kkbox_train_feature_v4.parquet")

In [ ]:
print("Loading V3 Data...")
df_v3 = pd.read_parquet(V3_PATH)
print(f"V3 Shape: {df_v3.shape}")

## 1. V3 기반 파생 변수 생성 (Arithmetic Derived Features)

In [ ]:
df_v4 = df_v3.copy()

# 1. Active Decay Rate (활동 감소율)
# Logic: (w7 active days) / (w30 active days / 4). If w30 is 0, set to 0 (or 1? 0 means no activity).
# epsilon to avoid division by zero
epsilon = 1e-6
df_v4['active_decay_rate'] = df_v4['num_days_active_w7'] / ((df_v4['num_days_active_w30'] / 4) + epsilon)

# 2. Listening Time Velocity (청취 가속도)
# Logic: w7 avg secs - w14 avg secs
df_v4['listening_time_velocity'] = df_v4['avg_secs_per_day_w7'] - df_v4['avg_secs_per_day_w14']

# 3. Discovery Index (탐색 지수)
# Logic: num_unq_w7 / num_songs_w7
df_v4['discovery_index'] = df_v4['num_unq_w7'] / (df_v4['num_songs_w7'] + epsilon)

# 4. Skip Passion Index (스킵 열정도)
# Logic: num_25_w7 / num_100_w7
df_v4['skip_passion_index'] = df_v4['num_25_w7'] / (df_v4['num_100_w7'] + epsilon)

# 5. Daily Listening Variance (Renaming existing feature for clarity)
df_v4['daily_listening_variance'] = df_v4['std_secs_w7']

# 6. Engagement Density (몰입 밀도)
# Logic: total_secs_w7 / num_days_active_w7. Note: This assumes avg_secs_per_day_w7 is calculated this way.
# Let's create it explicitly to be sure.
df_v4['engagement_density'] = df_v4['total_secs_w7'] / (df_v4['num_days_active_w7'] + epsilon)

print("Derived features created.")

## 2. Raw Log 기반 피처 생성 (Last Active Gap)

In [ ]:
print("Processing Raw User Logs for Last Active Gap...")
# user_logs_v2.csv is large, read necessary columns only
chunks = pd.read_csv(USER_LOGS_PATH, usecols=['msno', 'date'], chunksize=1000000)

last_active_df = pd.DataFrame()

# Find max date per user in chunks
max_dates = []
for chunk in chunks:
    chunk_max = chunk.groupby('msno')['date'].max()
    max_dates.append(chunk_max)

# Combine and find global max per user
all_max_dates = pd.concat(max_dates)
final_last_active = all_max_dates.groupby(level=0).max().reset_index()
final_last_active.rename(columns={'date': 'last_active_date'}, inplace=True)

# Convert to datetime
final_last_active['last_active_date'] = pd.to_datetime(final_last_active['last_active_date'], format='%Y%m%d')

print(f"Max Active Dates Calculated. Users: {len(final_last_active)}")

In [ ]:
# Determine Study Cutoff Date
# The training data (v3) is likely based on a specific month (e.g., March 2017).
# Let's check the max date in the logs to be consistent, or assume the end of the logs is the cutoff.
global_max_date = final_last_active['last_active_date'].max()
print(f"Global Max Date in Logs: {global_max_date}")

# Calculate Gap
final_last_active['last_active_gap'] = (global_max_date - final_last_active['last_active_date']).dt.days

print(final_last_active[['msno', 'last_active_gap']].head())

In [ ]:
# Merge with V4 DataFrame
df_v4 = df_v4.merge(final_last_active[['msno', 'last_active_gap']], on='msno', how='left')

# Fill NA for users with no logs (unlikely if they are in train set, but possible)
# If no log, gap is large number?? or -1?
# Assign a large number (e.g., 999) or the max gap found
max_gap_found = df_v4['last_active_gap'].max()
df_v4['last_active_gap'] = df_v4['last_active_gap'].fillna(max_gap_found + 1)

print("Merged Last Active Gap.")
print(df_v4[['msno', 'last_active_gap']].head())

## 3. 저장 (Save)

In [ ]:
print(f"Saving V4 to {OUTPUT_PATH}...")
df_v4.to_parquet(OUTPUT_PATH, index=False)
print("Done.")

In [ ]:
# Verify Features
print("New Feature Statistics:")
new_cols = ['active_decay_rate', 'listening_time_velocity', 'discovery_index', 'skip_passion_index', 'last_active_gap']
print(df_v4[new_cols].describe())